In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ------------------- -------------------- 6.6/13.3 MB 40.3 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 36.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/824.7 kB ? eta -:--:--
   --------------------------------------- 824.7/824.7 kB 18.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [4]:
import os
from dotenv import load_dotenv
import mlflow
import dagshub

# Load environment variables
load_dotenv()

# Initialize DagsHub tracking and link it to MLflow
dagshub.init(repo_owner="duddukuntamaneesh", repo_name="my-first-repo", mlflow=True)

# Set the MLflow Tracking URI via Dagshub initialization
print("✅ MLflow Tracking URI:", mlflow.get_tracking_uri())


INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as duddukuntamaneesh

INFO:dagshub:Accessing as duddukuntamaneesh
INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/duddukuntamaneesh/my-first-repo "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "duddukuntamaneesh/my-first-repo"

INFO:dagshub:Initialized MLflow to track repo "duddukuntamaneesh/my-first-repo"


Repository duddukuntamaneesh/my-first-repo initialized!

INFO:dagshub:Repository duddukuntamaneesh/my-first-repo initialized!


✅ MLflow Tracking URI: https://dagshub.com/duddukuntamaneesh/my-first-repo.mlflow


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow

# Initialize MLflow tracking
mlflow = set_mlflow_tracking()

# Load dataset
file_path = "../data/transformed/tabular_data.parquet"
df = pd.read_parquet(file_path, engine="pyarrow")

# Print available columns to identify the correct ones
print("Available columns:", df.columns)

# Define the correct datetime and target column names
datetime_col = "pickup_hour"  # Adjust if needed
target_col = "target"  # Adjust if needed

# Ensure the datetime column exists
if datetime_col not in df.columns:
    raise KeyError(f"❌ Column '{datetime_col}' not found! Check available columns: {df.columns}")

# Convert to datetime format if needed
df[datetime_col] = pd.to_datetime(df[datetime_col])

# Keep only relevant columns
df = df[[datetime_col, target_col]].dropna()

# Rename columns for Prophet
df.rename(columns={datetime_col: "ds", target_col: "y"}, inplace=True)

# Initialize and fit the Prophet model
model = Prophet()
model.fit(df)

# Generate future predictions
future = model.make_future_dataframe(periods=30, freq="H")  # Forecast for the next 30 hours
forecast = model.predict(future)

# Merge actual vs predicted values where available
merged = df.merge(forecast[['ds', 'yhat']], on='ds', how='inner')

# Compute Mean Absolute Error (MAE)
mae = mean_absolute_error(merged['y'], merged['yhat'])

# Print the MAE
print(f"📉 Mean Absolute Error (MAE): {mae:.2f}")

log_model_to_mlflow(model, df, "Prophet", "mean_absolute_error", score=mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.


Available columns: Index(['rides_t-672', 'rides_t-671', 'rides_t-670', 'rides_t-669',
       'rides_t-668', 'rides_t-667', 'rides_t-666', 'rides_t-665',
       'rides_t-664', 'rides_t-663',
       ...
       'rides_t-7', 'rides_t-6', 'rides_t-5', 'rides_t-4', 'rides_t-3',
       'rides_t-2', 'rides_t-1', 'pickup_hour', 'pickup_location_id',
       'target'],
      dtype='object', length=675)


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:Adding TBB (C:\Users\MD\AppData\Roaming\Python\Python312\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\MD\AppData\Local\Temp\tmpqh44k30_\9_201d5d.json
DEBUG:cmdstanpy:input tempfile: C:\Users\MD\AppData\Local\Temp\tmpqh44k30_\tg_ko3sv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\MD\\AppData\\Roaming\\Python\\Python312\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=4380', 'data', 'file=C:\\Users\\MD\\AppData\\Local\\Temp\\tmpqh44k30_\\9_201d5d.json', 'init=C:\\Users\\MD\\AppData\\Local\\Temp\\tmpqh44k30_\\tg_ko3sv.json', 'output', 'file=C:\\Users\

📉 Mean Absolute Error (MAE): 17.76


2025/03/05 16:31:59 INFO mlflow.tracking.fluent: Experiment with name 'Prophet' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: Prophet
INFO:src.experiment_utils:Logged mean_absolute_error: 17.75812719424949
C:\Users\MD\AppData\Roaming\Python\Python312\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details

Successfully registered model 'Prophet'.
2025/03/05 16:33:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Prophet, version 1
Created version '1' of model 'Prophet'.
INFO:src.experiment_utils:Model logged with name: Prophet


🏃 View run unruly-midge-982 at: https://dagshub.com/duddukuntamaneesh/my-first-repo.mlflow/#/experiments/6/runs/9e003c3996fa498aaac8042cf45751b9
🧪 View experiment at: https://dagshub.com/duddukuntamaneesh/my-first-repo.mlflow/#/experiments/6
